In [76]:
# We can scroll horizontally for the output
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

#Use pyspark to manipulate very large dataset
import pyspark
from pyspark.sql import SparkSession
# create spark object to import dataset
spark = SparkSession \
  .builder \
  .appName("Python Spark SQL basic example") \
  .config("spark.memory.fraction", 0.8) \
  .config("spark.executor.memory", "14g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.sql.shuffle.partitions" , "800") \
  .getOrCreate()
from pyspark.sql.window import Window  # Use window function
from pyspark.sql.functions import *  # max, sum, avg....
from pyspark.ml.feature import Imputer  # use imputer to deal with missing values

#pandas - they standard data manipulation package
import pandas as pd
#numpy - scientific computation - matrix operations, etc.
import numpy as np
#for feature selection
from sklearn import feature_selection
import os
from sklearn.preprocessing import MinMaxScaler # Normalization
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # Approximate missing values
from sklearn.feature_selection import VarianceThreshold # remove low variance features
from sklearn.feature_selection import f_classif # F test
from sklearn.model_selection import train_test_split

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 2000

# Combine Tables
Data cleaning, feature engineering

- Table 'credit_card_balance'
- Features used：SK_ID_CURR, MONTHS_BALANCE, AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL
- No missing value

In [3]:

# Import dataset and select demanded features
cash_balance = spark.read.format("csv").option("header", "true").load("credit_card_balance.csv")

df_cash_balance = cash_balance.select(col("SK_ID_CURR").cast('integer'), 
                                      col("MONTHS_BALANCE").cast('double'), 
                                      col("AMT_BALANCE").cast('double'),
                                      col("AMT_CREDIT_LIMIT_ACTUAL").cast('double'))

df_cash_balance = df_cash_balance.withColumn("pct_balance", col("AMT_BALANCE")/col("AMT_CREDIT_LIMIT_ACTUAL"))
print(df_cash_balance.dtypes)

cash_balance_feats=df_cash_balance.withColumn("pct_balance_0_1_year", when(col("MONTHS_BALANCE") >= -12, col("pct_balance"))) \
  .withColumn("pct_balance_1_2_year", when((col("MONTHS_BALANCE")<-12) & \
                                           (col("MONTHS_BALANCE")>=-24), col("pct_balance"))) \
  .withColumn("pct_balance_2+_year", when(col("MONTHS_BALANCE") < -24, col("pct_balance"))) \
  .groupBy("SK_ID_CURR") \
  .agg(sum("pct_balance").alias("pct_balance_total"),
       sum("pct_balance_0_1_year").alias("pct_balance_0_1_year"),
       sum("pct_balance_1_2_year").alias("pct_balance_1_2_year"),
       sum("pct_balance_2+_year").alias("pct_balance_2+_year"))

# Fill Null with 0
cash_balance_feats=cash_balance_feats.fillna(0, subset=['pct_balance_total', 'pct_balance_0_1_year', \
                                                        'pct_balance_1_2_year', 'pct_balance_2+_year'])
cash_balance_feats.show(10)

'''
# Pandas version: very slow ...
cash_balance = pd.read_csv ('credit_card_balance.csv')
cash_balance[["MONTHS_BALANCE", "AMT_BALANCE"]] = cash_balance[["MONTHS_BALANCE", "AMT_BALANCE"]].apply(pd.to_numeric)

cash_balance_feats = cash_balance.groupby('SK_ID_CURR').apply(lambda x: pd.Series({
      'amt_balance': x['AMT_BALANCE'].sum(),
      'amt_balance_0_1_year': x['AMT_BALANCE'][x['MONTHS_BALANCE'] >= -12].sum(),
      'amt_balance_1_2_ear': x['AMT_BALANCE'][(x['MONTHS_BALANCE'] < -12) & (x['MONTHS_BALANCE'] >= -24)].sum(),
      'amt_balance_2+_year': x['AMT_BALANCE'][(x['MONTHS_BALANCE'] < -24)].sum()
  })
)



'''

[('SK_ID_CURR', 'int'), ('MONTHS_BALANCE', 'double'), ('AMT_BALANCE', 'double'), ('AMT_CREDIT_LIMIT_ACTUAL', 'double'), ('pct_balance', 'double')]
+----------+--------------------+--------------------+--------------------+-------------------+
|SK_ID_CURR|   pct_balance_total|pct_balance_0_1_year|pct_balance_1_2_year|pct_balance_2+_year|
+----------+--------------------+--------------------+--------------------+-------------------+
|    357089|0.014510736842105264|0.014510736842105264|                 0.0|                0.0|
|    424333|                 0.0|                 0.0|                 0.0|                0.0|
|    343570|   43.58243300000001|   5.973389333333333|  11.940160666666667|          25.668883|
|    105665|   48.74947733333333|                 0.0|                 0.0|  48.74947733333333|
|    295286|          28.2322329|           12.278263|           8.7464449|           7.207525|
|    278515|           4.7070325|           3.7024989|           1.0045336|          

'\n# Pandas version: very slow ...\ncash_balance = pd.read_csv (\'credit_card_balance.csv\')\ncash_balance[["MONTHS_BALANCE", "AMT_BALANCE"]] = cash_balance[["MONTHS_BALANCE", "AMT_BALANCE"]].apply(pd.to_numeric)\n\ncash_balance_feats = cash_balance.groupby(\'SK_ID_CURR\').apply(lambda x: pd.Series({\n      \'amt_balance\': x[\'AMT_BALANCE\'].sum(),\n      \'amt_balance_0_1_year\': x[\'AMT_BALANCE\'][x[\'MONTHS_BALANCE\'] >= -12].sum(),\n      \'amt_balance_1_2_ear\': x[\'AMT_BALANCE\'][(x[\'MONTHS_BALANCE\'] < -12) & (x[\'MONTHS_BALANCE\'] >= -24)].sum(),\n      \'amt_balance_2+_year\': x[\'AMT_BALANCE\'][(x[\'MONTHS_BALANCE\'] < -24)].sum()\n  })\n)\n\n\n\n'

In [4]:
# Check if the above output is correct (select ID = 357089)
a = df_cash_balance.where(df_cash_balance["SK_ID_CURR"] == 357089).select("pct_balance")
a.select(sum("pct_balance")).collect()[0][0]

0.014510736842105264

- Table 'installments_payments': 
- Features used: SK_ID_CURR, DAYS_ENTRY_PAYMENT, DAYS_INSTALMENT, AMT_INSTALMENT, AMT_PAYMENT
- No missing value

In [5]:

# Import dataset and select demanded features
install_balance = spark.read.format("csv").option("header", "true").load("installments_payments.csv")

# Create features
df_install_balance = install_balance.withColumn("install_delay", col("DAYS_ENTRY_PAYMENT")-col("DAYS_INSTALMENT")) \
                                       .withColumn("install_default", col("AMT_INSTALMENT")-col("AMT_PAYMENT"))
    
df_install_balance = df_install_balance.select(col("SK_ID_CURR").cast('integer'), 
                                  col("install_delay").cast('double'), 
                                  col("install_default").cast('double'))

install_balance_feats=df_install_balance.withColumn("no_delay_default", when(col("install_delay") <= 0, col("install_default"))) \
  .withColumn("<=10d_default", when((col("install_delay")>0) & (col("install_delay")<=10), col("install_default"))) \
  .withColumn("10-20d_default", when((col("install_delay")>10) & (col("install_delay")<=20), col("install_default"))) \
  .withColumn("20-30d_default", when((col("install_delay")>20) & (col("install_delay")<=30), col("install_default"))) \
  .withColumn(">30d_default", when(col("install_delay")>30, col("install_default"))) \
  .groupBy("SK_ID_CURR") \
  .agg(sum("install_default").alias("install_default_total"),
       sum("no_delay_default").alias("no_delay_default"),
       sum("<=10d_default").alias("<=10d_default"),
       sum("10-20d_default").alias("10-20d_default"),
       sum("20-30d_default").alias("20-30d_default"),
       sum(">30d_default").alias(">30d_default"))

install_balance_feats=install_balance_feats.fillna(0, subset=["install_default_total", \
                                               "no_delay_default", "<=10d_default", "10-20d_default", \
                                                    "20-30d_default", ">30d_default"])

install_balance_feats.show(10)



+----------+---------------------+-------------------+------------------+------------------+----------------+------------+
|SK_ID_CURR|install_default_total|   no_delay_default|     <=10d_default|    10-20d_default|  20-30d_default|>30d_default|
+----------+---------------------+-------------------+------------------+------------------+----------------+------------+
|    145504|   29998.574999999997|          25602.435| 4396.140000000001|               0.0|             0.0|         0.0|
|    197588|                  0.0|                0.0|               0.0|               0.0|             0.0|         0.0|
|    154034|             18798.84|           16291.62|               0.0|2507.2200000000003|             0.0|         0.0|
|    101094|            27132.975|           27128.25| 4.725000000000364|               0.0|             0.0|         0.0|
|    137055|                  0.0|                0.0|               0.0|               0.0|             0.0|         0.0|
|    169588|    

In [6]:
# Check if the above output is correct
a = df_install_balance.where(df_install_balance["SK_ID_CURR"] == 145504).select("install_default")
a.select(sum("install_default")).collect()[0][0]

29998.574999999997

- Table 'POS_CASH_balance':
- Useful features:SK_DPD, SK_DPD_DEF
- Too few observations for these two features. Not gonna use this table

- Table 'bureau_balance':
- Too many observations missing. Not gonna use this table

- Table 'bureau':
- Features used: DAYS_CREDIT, AMT_CREDIT_MAX_OVERDUE, AMT_CREDIT_SUM, AMT_CREDIT_SUM_DEBT, AMT_CREDIT_SUM_LIMIT, AMT_ANNUITY

In [7]:

# Import dataset and select demanded features
bureau = spark.read.format("csv").option("header", "true").load("bureau.csv")

# Create features
df_bureau = bureau.select(col("SK_ID_CURR").cast('integer'), \
                                  col("DAYS_CREDIT").cast('double'), \
                                  col("AMT_CREDIT_MAX_OVERDUE").cast('double'), \
                                  col("AMT_CREDIT_SUM").cast('double'), \
                                  col("AMT_CREDIT_SUM_DEBT").cast('double'), \
                                  col("AMT_CREDIT_SUM_LIMIT").cast('double'), \
                                  col("AMT_ANNUITY").cast('double'))
df_bureau = df_bureau.withColumn("pct_credit-debit", (col("AMT_CREDIT_SUM")-col("AMT_CREDIT_SUM_DEBT")) \
                                                     /col("AMT_CREDIT_SUM_LIMIT"))

bureau_feats = df_bureau.groupby("SK_ID_CURR").agg(
      avg("DAYS_CREDIT").alias("avg_DAYS_CREDIT"),
      avg("AMT_CREDIT_MAX_OVERDUE").alias("avg_AMT_CREDIT_MAX_OVERDUE"),
      sum("pct_credit-debit").alias("pct_credit-debit_total"),
      sum("AMT_ANNUITY").alias("AMT_ANNUITY_total"))

bureau_feats.show(10)

+----------+-------------------+--------------------------+----------------------+-----------------+
|SK_ID_CURR|    avg_DAYS_CREDIT|avg_AMT_CREDIT_MAX_OVERDUE|pct_credit-debit_total|AMT_ANNUITY_total|
+----------+-------------------+--------------------------+----------------------+-----------------+
|    330299|            -1645.0|                   5496.56|                  null|         23906.07|
|    355377|            -1540.0|                       0.0|                  null|             null|
|    295286| -1264.888888888889|                   6191.37|                  null|        14999.985|
|    166160|             -856.0|                      null|                  null|             null|
|    191350|             -151.0|                      null|                  null|             null|
|    436175|-1871.2727272727273|        1804.5549999999998|     4.749633054619763|             null|
|    429364|            -2639.5|                    9265.5|                  null|         

In [8]:
# Check if the above output is correct
a = df_bureau.where(df_bureau["SK_ID_CURR"] == 295286).select("DAYS_CREDIT")
a.select(avg("DAYS_CREDIT")).collect()[0][0]

-1264.888888888889

In [9]:
# Check percentage of missing for each column

bureau_feats = bureau_feats.drop("pct_credit-debit_total") # Drop the column with more than 80% missing values

amount_missing_df = bureau_feats.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))) \
                                         .alias(c) for c in bureau_feats.columns])
amount_missing_df.show()

# Deal with missing values for multiple columns using imputer (replace with median)
imputer =  Imputer(inputCols=bureau_feats.columns, 
                   outputCols=["{}".format(c) for c in bureau_feats.columns],
                   strategy='median')
bureau_feats = imputer.fit(bureau_feats).transform(bureau_feats)

bureau_feats.show(10)

+----------+---------------+--------------------------+------------------+
|SK_ID_CURR|avg_DAYS_CREDIT|avg_AMT_CREDIT_MAX_OVERDUE| AMT_ANNUITY_total|
+----------+---------------+--------------------------+------------------+
|       0.0|            0.0|        0.3035862019351822|0.6134082815856853|
+----------+---------------+--------------------------+------------------+

+----------+-------------------+--------------------------+-----------------+
|SK_ID_CURR|    avg_DAYS_CREDIT|avg_AMT_CREDIT_MAX_OVERDUE|AMT_ANNUITY_total|
+----------+-------------------+--------------------------+-----------------+
|    330299|            -1645.0|                   5496.56|         23906.07|
|    355377|            -1540.0|                       0.0|          20836.8|
|    295286| -1264.888888888889|                   6191.37|        14999.985|
|    166160|             -856.0|                       0.0|          20836.8|
|    191350|             -151.0|                       0.0|          20836.8|


- Table 'previous_application':
- Features used: AMT_ANNUITY, AMT_CREDIT, AMT_GOODS_PRICE, NAME_YIELD_GROUP, NFLAG_INSURED_ON_APPROVAL

In [10]:
def find_mode(values_list):
    try:
        mode = max(set(values_list), key=values_list.count) #get the median of values in a list in each row
        return mode
    except Exception:
        return None #if there is anything wrong with the given values

mode_finder = udf(find_mode)

In [11]:
# Import dataset and select demanded features
prev_application = spark.read.format("csv").option("header", "true").load("previous_application.csv")

# Select features
df_prev_application = prev_application.select(col("SK_ID_CURR").cast('integer'), \
                                  col("AMT_ANNUITY").cast('double'), \
                                  col("AMT_CREDIT").cast('double'), \
                                  col("AMT_GOODS_PRICE").cast('double'), \
                                  col("NAME_YIELD_GROUP"), \
                                  col("NFLAG_INSURED_ON_APPROVAL").cast('integer'))

df_prev_application = df_prev_application.withColumn("label_YIELD_GROUP", \
              when((df_prev_application["NAME_YIELD_GROUP"] == 'low_action') | \
                   (df_prev_application["NAME_YIELD_GROUP"] == 'low_normal'), 1).otherwise( \
              when(df_prev_application["NAME_YIELD_GROUP"] == 'middle', 2).otherwise( \
              when(df_prev_application["NAME_YIELD_GROUP"] == 'high', 3).otherwise(lit(None)))))

# Check percentage of missing for each column
amount_missing_df = df_prev_application.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))) \
                                         .alias(c) for c in df_prev_application.columns])
amount_missing_df.show()


+----------+-------------------+--------------------+-------------------+----------------+-------------------------+------------------+
|SK_ID_CURR|        AMT_ANNUITY|          AMT_CREDIT|    AMT_GOODS_PRICE|NAME_YIELD_GROUP|NFLAG_INSURED_ON_APPROVAL| label_YIELD_GROUP|
+----------+-------------------+--------------------+-------------------+----------------+-------------------------+------------------+
|       0.0|0.22286665062081865|5.987256722791211E-7|0.23081772754868538|             0.0|      0.40298129461254667|0.3096698985878456|
+----------+-------------------+--------------------+-------------------+----------------+-------------------------+------------------+



In [12]:
# Deal with missing values for multiple columns using imputer (replace with median)
df_prev_application = df_prev_application.drop("NAME_YIELD_GROUP")
imputer =  Imputer(inputCols=df_prev_application.columns, 
                   outputCols=["{}".format(c) for c in df_prev_application.columns],
                   strategy='median')
df_prev_application = imputer.fit(df_prev_application).transform(df_prev_application)

In [13]:

# Create new features that are the mode for "label_YIELD_GROUP" and "NFLAG_INSURED_ON_APPROVAL"
grouped = df_prev_application.groupBy("SK_ID_CURR", "label_YIELD_GROUP").count()
window = Window.partitionBy("SK_ID_CURR").orderBy(desc("count"))
df_yield = grouped \
    .withColumn("order", row_number().over(window)) \
    .where(col("order") == 1) \
    .select(col("SK_ID_CURR"), col("label_YIELD_GROUP").alias("prev_label_YIELD_mode"))


grouped = df_prev_application.groupBy("SK_ID_CURR", "NFLAG_INSURED_ON_APPROVAL").count()
window = Window.partitionBy("SK_ID_CURR").orderBy(desc("count"))
df_insured = grouped \
    .withColumn("order", row_number().over(window)) \
    .where(col("order") == 1) \
    .select(col("SK_ID_CURR"), col("NFLAG_INSURED_ON_APPROVAL").alias("prev_NFLAG_INSURED_mode"))

# Create other features                                                                      
prev_application_feats = df_prev_application.groupby("SK_ID_CURR").agg(
      sum("AMT_ANNUITY").alias("prev_AMT_ANNUITY_total"),
      sum("AMT_CREDIT").alias("prev_AMT_CREDIT_total"),
      sum("AMT_GOODS_PRICE").alias("prev_GOODS_price_total"))

# Left join all features                                                                    
prev_application_feats = prev_application_feats.join(df_yield, on = 'SK_ID_CURR', how = 'left')
prev_application_feats = prev_application_feats.join(df_insured, on = 'SK_ID_CURR', how = 'left')

prev_application_feats.show(10)

+----------+----------------------+---------------------+----------------------+---------------------+-----------------------+
|SK_ID_CURR|prev_AMT_ANNUITY_total|prev_AMT_CREDIT_total|prev_GOODS_price_total|prev_label_YIELD_mode|prev_NFLAG_INSURED_mode|
+----------+----------------------+---------------------+----------------------+---------------------+-----------------------+
|    420485|     74637.04500000001|             640035.0|              863203.5|                    2|                      0|
|    239142|             233017.38|            4441855.5|             4291650.0|                    2|                      0|
|    201148|            140416.695|            1815498.0|             1781370.0|                    3|                      0|
|    397549|    192520.03499999997|            2653483.5|            2572793.73|                    2|                      0|
|    251529|             56862.405|             778959.0|              463095.0|                    2|         

In [14]:
# Check if the above output is correct
a = df_prev_application.where(df_prev_application["SK_ID_CURR"] == 397549).select("AMT_GOODS_PRICE")
a.select(sum("AMT_GOODS_PRICE")).collect()[0][0]

2572793.73

In [15]:
# Check total number of insured clients (is this feature significant?)
prev_application_feats.select(sum("prev_NFLAG_INSURED_mode")).collect()[0][0]

39275

- Table 'application_train': 
- data cleaning, combine tables, feature selection 
- Use pandas below because training set has fewer observations so have relatively higher running speed, \
also pyspark doesn't have an efficient function to do one-hot encoding)

In [42]:
# Import the training set
train = pd.read_csv('application_train.csv')

print(train.dtypes)

SK_ID_CURR                        int64
TARGET                            int64
NAME_CONTRACT_TYPE               object
CODE_GENDER                      object
FLAG_OWN_CAR                     object
FLAG_OWN_REALTY                  object
CNT_CHILDREN                      int64
AMT_INCOME_TOTAL                float64
AMT_CREDIT                      float64
AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
NAME_TYPE_SUITE                  object
NAME_INCOME_TYPE                 object
NAME_EDUCATION_TYPE              object
NAME_FAMILY_STATUS               object
NAME_HOUSING_TYPE                object
REGION_POPULATION_RELATIVE      float64
DAYS_BIRTH                        int64
DAYS_EMPLOYED                     int64
DAYS_REGISTRATION               float64
DAYS_ID_PUBLISH                   int64
OWN_CAR_AGE                     float64
FLAG_MOBIL                        int64
FLAG_EMP_PHONE                    int64
FLAG_WORK_PHONE                   int64


Categorical features: NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE
NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, FONDKAPREMONT_MODE
HOUSETYPE_MODE, WALLSMATERIAL_MODE, EMERGENCYSTATE_MODE, NAME_EDUCATION_TYPE

In [43]:
# Split response and features
y = train['TARGET']
X = train.drop(['TARGET'],axis=1)

In [44]:
#one-hot encode the categorical variables
X_factorized = pd.get_dummies(X)  # Convert all non-numeric variables to numeric

#clean up infinite values also - just in case
X_factorized.replace([np.inf, -np.inf], np.nan)

X_factorized.head(10)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0,0,0,0,0,0
5,100008,0,99000.0,490495.5,27517.5,454500.0,0.035792,-16941,-1588,-4970.0,...,0,0,0,0,0,0,0,0,0,0
6,100009,1,171000.0,1560726.0,41301.0,1395000.0,0.035792,-13778,-3130,-1213.0,...,0,0,0,0,0,0,0,0,0,0
7,100010,0,360000.0,1530000.0,42075.0,1530000.0,0.003122,-18850,-449,-4597.0,...,0,0,0,0,0,0,0,0,0,0
8,100011,0,112500.0,1019610.0,33826.5,913500.0,0.018634,-20099,365243,-7427.0,...,0,0,0,0,0,0,0,0,0,0
9,100012,0,135000.0,405000.0,20250.0,405000.0,0.019689,-14469,-2019,-14437.0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# Delete the features that are not in the test set
X_factorized = X_factorized.drop(['CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown'], \
                                 axis=1)
print(len(X_factorized.columns))

242


In [46]:
#Downsample our data (stratified) for speed 
train_factorized = pd.concat([X_factorized, y], axis=1)
train_downsample = train_factorized[train_factorized['TARGET'] ==1]
train_downsample = train_downsample.append(train_factorized[train_factorized['TARGET'] ==0] \
                                           .sample(frac = 0.1, replace = False))
print(train_factorized.shape, train_downsample.shape)

(307511, 243) (53094, 243)


Now we have the final training set before removing any insignificant features \
We want to merge this training set with the 4 pyspark tables that we generated previously to have all features together

In [47]:
# Convert from pandas to pyspark

train = spark.createDataFrame(train_downsample)

train.show(10)

+----------+------------+----------------+----------+-----------+---------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+---------------+-----------------+----------------------------+----------------+-------------------+--------------+--------------+--------------+--------------+-------------+---------------------+-----

In [48]:
# Join the all the feature tables with the training dataset

train = train.join(cash_balance_feats, on = 'SK_ID_CURR', how = 'left')
train = train.join(install_balance_feats, on = 'SK_ID_CURR', how = 'left')
train = train.join(bureau_feats, on = 'SK_ID_CURR', how = 'left')
train = train.join(prev_application_feats, on = 'SK_ID_CURR', how = 'left')

train.show(10)

+----------+------------+----------------+----------+-----------+---------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+-------------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+-----

In [49]:
# Convert the final pyspark table back to pandas to do variable screening (pandas is more convenient)
train = train.toPandas()

# It takes around 5 min to run this line on my pc ...

train.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,20-30d_default,>30d_default,avg_DAYS_CREDIT,avg_AMT_CREDIT_MAX_OVERDUE,AMT_ANNUITY_total,prev_AMT_ANNUITY_total,prev_AMT_CREDIT_total,prev_GOODS_price_total,prev_label_YIELD_mode,prev_NFLAG_INSURED_mode
0,101552,0,126000.0,566055.0,16681.5,472500.0,0.028663,-18311,-4927,-4593.0,...,0.0,0.0,-1643.000000,1300.78125,20836.8,1906.515,17523.0,17716.5,3.0,0.0
1,113770,1,49500.0,652500.0,21046.5,652500.0,0.007120,-15291,-3378,-477.0,...,0.0,0.0,-1434.285714,531.61875,36360.0,3639.060,33115.5,33115.5,1.0,0.0
2,117180,0,135000.0,835605.0,24561.0,697500.0,0.010556,-19589,-424,-2267.0,...,0.0,0.0,-1008.375000,0.00000,20836.8,39046.500,830655.0,722529.0,1.0,0.0
3,124966,1,126000.0,495000.0,30411.0,495000.0,0.006852,-12332,-3747,-2530.0,...,0.0,0.0,-1750.500000,0.00000,20836.8,6974.775,112275.0,136800.0,1.0,0.0
4,136815,0,90000.0,454500.0,15151.5,454500.0,0.006296,-23781,365243,-14303.0,...,0.0,0.0,-344.333333,0.00000,20836.8,55439.415,339741.0,628515.0,2.0,0.0


In [50]:
# checking missing data
y = train['TARGET']
X_miss = train.drop(['TARGET'],axis=1)
total_null = X_miss.isnull().count()
percent = (X_miss.isnull().sum()/X_miss.isnull().count()*100)
missing_check  = pd.concat([total_null, percent], axis=1, keys=['Total Null', 'Percent']).sort_values(ascending = False, by = ['Percent'])
print(missing_check)

                                                   Total Null    Percent
COMMONAREA_MEDI                                         53094  71.644630
COMMONAREA_MODE                                         53094  71.644630
COMMONAREA_AVG                                          53094  71.644630
NONLIVINGAPARTMENTS_MEDI                                53094  71.254756
NONLIVINGAPARTMENTS_AVG                                 53094  71.254756
NONLIVINGAPARTMENTS_MODE                                53094  71.254756
pct_balance_0_1_year                                    53094  70.751874
pct_balance_total                                       53094  70.751874
pct_balance_2+_year                                     53094  70.751874
pct_balance_1_2_year                                    53094  70.751874
LIVINGAPARTMENTS_MODE                                   53094  70.281011
LIVINGAPARTMENTS_AVG                                    53094  70.281011
LIVINGAPARTMENTS_MEDI                              

In [51]:
# Approximate missing values using imputer (for all features)

# Use iterative imputer
#imputer = IterativeImputer(random_state=0, initial_strategy='median')
#train_factorized = pd.DataFrame(imputer.fit_transform(train_factorized),columns = train_factorized.columns)

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
X_complete = pd.DataFrame(imputer.fit_transform(X_miss),columns = X_miss.columns)
X_complete.head(10)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,20-30d_default,>30d_default,avg_DAYS_CREDIT,avg_AMT_CREDIT_MAX_OVERDUE,AMT_ANNUITY_total,prev_AMT_ANNUITY_total,prev_AMT_CREDIT_total,prev_GOODS_price_total,prev_label_YIELD_mode,prev_NFLAG_INSURED_mode
0,101552.0,0.0,126000.0,566055.0,16681.5,472500.0,0.028663,-18311.0,-4927.0,-4593.0,...,0.0,0.0,-1643.000000,1300.78125,20836.8,1906.515,17523.0,17716.5,3.0,0.0
1,113770.0,1.0,49500.0,652500.0,21046.5,652500.0,0.007120,-15291.0,-3378.0,-477.0,...,0.0,0.0,-1434.285714,531.61875,36360.0,3639.060,33115.5,33115.5,1.0,0.0
2,117180.0,0.0,135000.0,835605.0,24561.0,697500.0,0.010556,-19589.0,-424.0,-2267.0,...,0.0,0.0,-1008.375000,0.00000,20836.8,39046.500,830655.0,722529.0,1.0,0.0
3,124966.0,1.0,126000.0,495000.0,30411.0,495000.0,0.006852,-12332.0,-3747.0,-2530.0,...,0.0,0.0,-1750.500000,0.00000,20836.8,6974.775,112275.0,136800.0,1.0,0.0
4,136815.0,0.0,90000.0,454500.0,15151.5,454500.0,0.006296,-23781.0,365243.0,-14303.0,...,0.0,0.0,-344.333333,0.00000,20836.8,55439.415,339741.0,628515.0,2.0,0.0
5,137147.0,0.0,135000.0,180000.0,17932.5,180000.0,0.015221,-17721.0,-775.0,-6875.0,...,0.0,0.0,-829.166667,0.00000,20836.8,69227.955,953991.0,1154322.0,2.0,0.0
6,142956.0,2.0,202500.0,1305000.0,38286.0,1305000.0,0.010147,-13316.0,-1231.0,-7035.0,...,0.0,0.0,-2089.400000,0.00000,20836.8,115061.580,1145340.0,1416150.0,2.0,0.0
7,143916.0,0.0,157500.0,781920.0,32998.5,675000.0,0.035792,-16410.0,-595.0,-1587.0,...,0.0,0.0,-451.166667,0.00000,13117.5,69327.810,1584036.0,1609042.5,1.0,0.0
8,169522.0,0.0,108000.0,180000.0,9000.0,180000.0,0.010966,-7712.0,-264.0,-3828.0,...,0.0,0.0,-73.800000,0.00000,20836.8,17507.700,160092.0,160092.0,1.0,0.0
9,171935.0,2.0,123750.0,382500.0,19125.0,382500.0,0.046220,-12722.0,-1374.0,-3821.0,...,0.0,0.0,-963.800000,0.00000,20836.8,29107.170,31689.0,267255.0,2.0,0.0


Now we have a clean and complete training set. \
Next we need to do variable screening (feature selection)

In [52]:
# Check the variance for all features
print(X_complete.var().sort_values(kind="quicksort",ascending=True))

NAME_INCOME_TYPE_Student                             0.000000e+00
FLAG_DOCUMENT_12                                     0.000000e+00
FLAG_MOBIL                                           0.000000e+00
NAME_INCOME_TYPE_Businessman                         1.883452e-05
FLAG_DOCUMENT_10                                     3.766833e-05
FLAG_DOCUMENT_2                                      7.533382e-05
FLAG_DOCUMENT_4                                      7.533382e-05
ORGANIZATION_TYPE_Industry: type 8                   9.416550e-05
ORGANIZATION_TYPE_Trade: type 5                      1.129965e-04
ORGANIZATION_TYPE_Trade: type 4                      1.129965e-04
FLAG_DOCUMENT_7                                      1.506563e-04
FLAG_DOCUMENT_17                                     1.694851e-04
REGION_POPULATION_RELATIVE                           1.720488e-04
NAME_INCOME_TYPE_Unemployed                          2.071407e-04
ORGANIZATION_TYPE_Religion                           2.636187e-04
NAME_EDUCA

In [53]:
# Remove features with vairance lower than 
selector = VarianceThreshold(threshold=0.001)
selector.fit(X_complete)
X_var = X_complete[X_complete.columns[selector.get_support(indices=True)]]

len(X_var.columns)

229

In [62]:
# Calculate F Statistic and the corresponding p values
F_stat, p_value = f_classif(X_var, y)

# convert to a DF
ANOVA = pd.DataFrame(data = {'Features': X_var.columns.values, \
                'F-value': F_stat, 'p-value': p_value.round(decimals=10)})
ANOVA.sort_values(by = ['F-value'], ascending = False, ignore_index=True, inplace = True)
print(ANOVA)

# select the features with F-value > 3.95
index = ANOVA['F-value']>3.95
X_sel=X_var[ANOVA['Features'][index]]

                                              Features      F-value  \
0                                         EXT_SOURCE_3  4363.283099   
1                                         EXT_SOURCE_2  4241.002720   
2                                         EXT_SOURCE_1  1867.954989   
3                                      avg_DAYS_CREDIT  1238.394950   
4                                           DAYS_BIRTH  1141.815937   
5                          REGION_RATING_CLIENT_W_CITY   716.635675   
6                                 REGION_RATING_CLIENT   673.994964   
7                 NAME_EDUCATION_TYPE_Higher education   635.488809   
8                             NAME_INCOME_TYPE_Working   590.002150   
9                                 pct_balance_0_1_year   558.110440   
10                              DAYS_LAST_PHONE_CHANGE   543.963536   
11                                       CODE_GENDER_F   543.473735   
12                                       CODE_GENDER_M   543.473735   
13   N

In [73]:
# Remove features that might cause multicollinearity issues
train_cov = pd.concat([X_sel, y], axis=1,sort=False)
cor = train_cov.corr().abs()
# Select upper/lower triangle of correlation matrix
upper_triangle = cor.where(np.triu(np.ones(cor.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.85
drop_lst1 = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.85)]

print('number of features dropped that can cause multicollinearity: ' + str(len(drop_lst1)))
# Drop features 
train_cov.drop(drop_lst1, axis=1, inplace=True)
print('number of features left: ' + str(len(train_cov.columns)))

number of features dropped that can cause multicollinearity: 44
number of features left: 127


Now we have the finalized training set \
Gonna split the training set into training + valiadation

In [213]:
# Normalized X and y
y = train_cov['TARGET']
X = train_cov.drop(['TARGET'],axis=1)

sc = MinMaxScaler()
X = pd.DataFrame(sc.fit_transform(X), columns=X.columns)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

We are ready to do model fitting 

For the above training and validation sets, we are gonna fit models, do hyperparameter tuning, do cross-evaluation \
on different models to compare the model performance, then select the best model. Then we will use the best model \
to predict for the ultimate test set

Notice we will do further feature engineering and feature selection before we fit the logistic regression model \
(maybe do clustering, stepwise selection and WOE transformation)

Below we will deal with the test set

- Table 'application_test'
- Data cleaning, combine table, and feature selection based on what we did for the training set
- test set doesn't have a reponse variable

In [78]:
# Import the test set
test = pd.read_csv('application_test.csv')

test.head(10)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Marr

In [79]:
#one-hot encode the categorical variables
test_factorized = pd.get_dummies(test)  # Convert all non-numeric variables to numeric

#clean up infinite values also - just in case
test_factorized.replace([np.inf, -np.inf], np.nan)

test_factorized.head(10)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business 

In [80]:
# Check if test set has same features as the training set
# If not, delete those features from the traning's features
lst = []
for c in X_factorized.columns:
    if c not in test_factorized.columns:
        lst.append(c)
print(lst)

[]


In [81]:
# Convert from pandas to pyspark

test = spark.createDataFrame(test_factorized)

test.show(10)

+----------+------------+----------------+----------+-----------+---------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-------------------+-------------------+------------------+--------------+--------------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+------

In [82]:
# Join the all the feature tables with the test dataset

test = test.join(cash_balance_feats, on = 'SK_ID_CURR', how = 'left')
test = test.join(install_balance_feats, on = 'SK_ID_CURR', how = 'left')
test = test.join(bureau_feats, on = 'SK_ID_CURR', how = 'left')
test = test.join(prev_application_feats, on = 'SK_ID_CURR', how = 'left')

test.show(10)

+----------+------------+----------------+----------+-----------+---------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+--------------------+---------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------

In [83]:
# Convert the final pyspark table back to pandas to do variable screening (pandas is more convenient)
test = test.toPandas()

# It takes around 5 min to run this line on my pc ...

test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business 

In [85]:
# Approximate missing values using imputer (fit on the training dataset and apply on the test set)

# Use iterative imputer
#imputer = IterativeImputer(random_state=0, initial_strategy='median')
#train_factorized = pd.DataFrame(imputer.fit_transform(train_factorized),columns = train_factorized.columns)
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
test_complete = pd.DataFrame(imputer.fit(X_miss).transform(test),columns = test.columns)
test_complete.head(10)

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business 

In [89]:
# The finalized test set for model prediction after normalization
# Note that we would do further feature selection and feature engineering before fitting the logistic model
test = test_complete[X.columns]

sc = MinMaxScaler()
test = pd.DataFrame(sc.fit_transform(test), columns=test.columns)
test.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,avg_DAYS_CREDIT,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,NAME_EDUCATION_TYPE_Higher education,NAME_INCOME_TYPE_Working,pct_balance_0_1_year,DAYS_LAST_PHONE_CHANGE,CODE_GENDER_F,DAYS_ID_PUBLISH,NAME_INCOME_TYPE_Pensioner,REG_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_3,DAYS_REGISTRATION,EMERGENCYSTATE_MODE_No,FLOORSMAX_AVG,AMT_GOODS_PRICE,OCCUPATION_TYPE_Laborers,ELEVATORS_AVG,prev_label_YIELD_mode,REG_CITY_NOT_LIVE_CITY,WALLSMATERIAL_MODE_Panel,TOTALAREA_MODE,NAME_CONTRACT_TYPE_Cash loans,LIVE_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_6,DEF_30_CNT_SOCIAL_CIRCLE,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_With parents,install_default_total,OCCUPATION_TYPE_Drivers,ORGANIZATION_TYPE_Self-employed,FLAG_WORK_PHONE,HOUR_APPR_PROCESS_START,NAME_FAMILY_STATUS_Single / not married,FLAG_PHONE,NAME_INCOME_TYPE_State servant,NAME_FAMILY_STATUS_Married,OCCUPATION_TYPE_Low-skill Laborers,LIVINGAPARTMENTS_AVG,FLOORSMIN_AVG,OCCUPATION_TYPE_Accountants,FONDKAPREMONT_MODE_reg oper account,ORGANIZATION_TYPE_Business Entity Type 3,NAME_FAMILY_STATUS_Civil marriage,OCCUPATION_TYPE_Core staff,BASEMENTAREA_AVG,NONLIVINGAREA_AVG,NAME_FAMILY_STATUS_Widow,FLAG_OWN_CAR_Y,OCCUPATION_TYPE_Sales staff,COMMONAREA_AVG,CNT_CHILDREN,OCCUPATION_TYPE_Managers,AMT_REQ_CREDIT_BUREAU_YEAR,ORGANIZATION_TYPE_Construction,ENTRANCES_AVG,NAME_HOUSING_TYPE_Rented apartment,OWN_CAR_AGE,OCCUPATION_TYPE_Security staff,pct_balance_1_2_year,AMT_REQ_CREDIT_BUREAU_MON,FLAG_DOCUMENT_13,AMT_ANNUITY,ORGANIZATION_TYPE_Transport: type 3,FLAG_DOCUMENT_14,OCCUPATION_TYPE_High skill tech staff,LANDAREA_MEDI,ORGANIZATION_TYPE_School,FLAG_DOCUMENT_16,prev_NFLAG_INSURED_mode,FONDKAPREMONT_MODE_reg oper spec account,pct_balance_2+_year,YEARS_BUILD_MODE,WALLSMATERIAL_MODE_Monolithic,"WALLSMATERIAL_MODE_Stone, brick",FONDKAPREMONT_MODE_org spec account,NAME_INCOME_TYPE_Commercial associate,ORGANIZATION_TYPE_Police,>30d_default,OBS_30_CNT_SOCIAL_CIRCLE,ORGANIZATION_TYPE_Military,OCCUPATION_TYPE_Private service staff,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Bank,NAME_TYPE_SUITE_Family,OCCUPATION_TYPE_Medicine staff,prev_GOODS_price_total,AMT_REQ_CREDIT_BUREAU_QRT,NAME_EDUCATION_TYPE_Lower secondary,OCCUPATION_TYPE_Waiters/barmen staff,NAME_TYPE_SUITE_Unaccompanied,FLAG_DOCUMENT_8,<=10d_default,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Trade: type 7,OCCUPATION_TYPE_Cooking staff,FLAG_DOCUMENT_18,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Industry: type 12,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_Block,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Kindergarten,REG_REGION_NOT_WORK_REGION,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_Industry: type 9,WEEKDAY_APPR_PROCESS_START_TUESDAY,ORGANIZATION_TYPE_Industry: type 1,WEEKDAY_APPR_PROCESS_START_MONDAY,avg_AMT_CREDIT_MAX_OVERDUE,YEARS_BEGINEXPLUATATION_AVG,ORGANIZATION_TYPE_Agriculture,OCCUPATION_TYPE_Cleaning staff,WEEKDAY_APPR_PROCESS_START_SATURDAY,ORGANIZATION_TYPE_Insurance,NAME_TYPE_SUITE_Other_B,ORGANIZATION_TYPE_Industry: type 4,FLAG_OWN_REALTY_Y,HOUSETYPE_MODE_specific housing
0,0.237916,0.571620,0.488005,-1153.250000,-11684.0,2.0,0.0,0.0,0.073774,-2099.0,0.0,-4197.0,0.0,0.0,1.0,-3559.0,0.0,0.1667,225000.0,0.0,0.0,2.0,0.0,0.0,0.0669,1.0,0.0,0.0,0.0,0.0,1.0,0.000,0.0,0.0,0.0,14.0,0.0,1.0,0.0,1.0,0.0,0.0740,0.2083,0.0,0.0,1.0,0.0,0.0,0.0754,0.0034,0.0,1.0,0.0,0.0204,1.0,0.0,1.0,0.0,0.1379,0.0,11.0,0.0,0.000000,0.0,0.0,25447.5,0.0,0.0,0.0,0.0472,0.0,0.0,0.0,0.0,0.0,0.7583,0.0,0.0,0.0,1.0,0.0,0.000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,172755.0,1.0,0.0,0.0,1.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4862.88,0.9816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.255332,0.564193,0.733277,-779.500000,-20541.0,2.0,0.0,0.0,10.783308,-739.0,1.0,-3671.0,1.0,0.0,1.0,-9452.0,0.0,0.1667,148500.0,0.0,0.0,2.0,0.0,0.0,0.0669,1.0,0.0,0.0,0.0,1.0,0.0

- Additional Feature Engineering for Logistic Regression

Deal with training set

In [90]:
# Stepwise logistic regression to do further feature selection
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):

    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        pv_list = pd.DataFrame(columns=('VAR_name', 'P-value'))
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval = pd.Series([new_column, model.pvalues[new_column]], index=pv_list.columns)
            pv_list = pv_list.append(new_pval,ignore_index=True)
        best_pval = pv_list['P-value'].min()
        if best_pval < threshold_in:
            best_feature = pv_list['VAR_name'].iloc[pv_list['P-value'].idxmin(axis=1)]
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = included[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X, y)
print('resulting features:')
print(result)

Optimization terminated successfully.
         Current function value: 0.690137
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690261
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.690931
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690775
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690937
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690780
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687597
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690971
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690398
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690214
  

Optimization terminated successfully.
         Current function value: 0.690504
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.689874
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.690929
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.686932
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690738
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688204
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690722
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690342
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.687090
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.690910
  

Optimization terminated successfully.
         Current function value: 0.651083
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.645693
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651476
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651696
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651648
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651633
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.650962
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.647066
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.650970
  

Add EXT_SOURCE_2                   with p-value 0.0
Optimization terminated successfully.
         Current function value: 0.621526
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621028
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621267
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621515
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621346
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621521
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621508
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.619131
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621450
         Iterations 5
Optimization terminated succes

Optimization terminated successfully.
         Current function value: 0.621488
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.619338
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621299
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.619705
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621408
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621249
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.620131
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621391
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621306
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.621320
  

...

Optimization terminated successfully.
         Current function value: 0.591619
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591637
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591635
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591638
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591609
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591629
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591612
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591584
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591625
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591603
  

Optimization terminated successfully.
         Current function value: 0.591550
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591566
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591566
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591566
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591568
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591554
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591552
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591566
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591525
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591509
  

Optimization terminated successfully.
         Current function value: 0.591487
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591472
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591470
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591484
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591450
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591427
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591488
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591457
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591457
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591474
  

Optimization terminated successfully.
         Current function value: 0.591317
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591379
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591349
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591350
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591365
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591370
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591374
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591378
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591326
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591372
  

Optimization terminated successfully.
         Current function value: 0.591306
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591310
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591258
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591304
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591277
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591311
         Iterations 6
Add ORGANIZATION_TYPE_Security Ministries with p-value 0.00967953
Optimization terminated successfully.
         Current function value: 0.591247
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591246
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.591148
         Iterations 7
Optimization ter

In [91]:
# Final selected variables
features_lst = pd.DataFrame({"variables":result})
print(features_lst)
print('final number of variables: ' + str(len(features_lst)))

                                   variables
0                               EXT_SOURCE_3
1                               EXT_SOURCE_2
2                               EXT_SOURCE_1
3                       pct_balance_0_1_year
4       NAME_EDUCATION_TYPE_Higher education
5                              CODE_GENDER_F
6                            FLAG_DOCUMENT_3
7                             FLAG_OWN_CAR_Y
8                            DAYS_ID_PUBLISH
9                REGION_RATING_CLIENT_W_CITY
10                  DEF_30_CNT_SOCIAL_CIRCLE
11                     prev_label_YIELD_mode
12                  NAME_INCOME_TYPE_Working
13                               AMT_ANNUITY
14                NAME_FAMILY_STATUS_Married
15                  WALLSMATERIAL_MODE_Panel
16                    DAYS_LAST_PHONE_CHANGE
17                     install_default_total
18                 AMT_REQ_CREDIT_BUREAU_QRT
19                    prev_GOODS_price_total
20           ORGANIZATION_TYPE_Self-employed
21  ORGANI

In [214]:
# Select the features form the stepwise selection
X_logit = X[result].copy()

In [215]:
# WOE transformation. Deal with categorical and cts variables separately
print(X_logit.nunique())
index_categ = X_logit.nunique()<7
index_numeric = X_logit.nunique()>=7
df_categ = X_logit[X_logit.columns[index_categ]]
df_numeric = X_logit[X_logit.columns[index_numeric]]

print(df_categ.nunique())
print('number of continuous features: ' + str(len(df_numeric.columns)))
print('number of categorical features: ' + str(len(df_categ.columns)))

EXT_SOURCE_3                                  776
EXT_SOURCE_2                                42307
EXT_SOURCE_1                                21926
pct_balance_0_1_year                         8046
NAME_EDUCATION_TYPE_Higher education            2
CODE_GENDER_F                                   2
FLAG_DOCUMENT_3                                 2
FLAG_OWN_CAR_Y                                  2
DAYS_ID_PUBLISH                              5744
REGION_RATING_CLIENT_W_CITY                     3
DEF_30_CNT_SOCIAL_CIRCLE                        7
prev_label_YIELD_mode                           3
NAME_INCOME_TYPE_Working                        2
AMT_ANNUITY                                  8746
NAME_FAMILY_STATUS_Married                      2
WALLSMATERIAL_MODE_Panel                        2
DAYS_LAST_PHONE_CHANGE                       3366
install_default_total                       24738
AMT_REQ_CREDIT_BUREAU_QRT                      10
prev_GOODS_price_total                      41999


In [216]:
# WOE transformation

# Categorical variables
for c in df_categ.columns:
    col_binned = df_categ[c].copy()
    cross = pd.crosstab(col_binned, y)
    for i in range(len(cross.index)):
        # if some bins have 0 event, use adjust WOE by adding 0.5
        if cross.iloc[i,0] == 0 or cross.iloc[i,1] == 0:
            non_default = (cross.iloc[i,0]+0.5)/cross.iloc[:,0].values.sum()
            default = (cross.iloc[i,1]+0.5)/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        else:
            non_default = cross.iloc[i,0]/cross.iloc[:,0].values.sum()
            default = cross.iloc[i,1]/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        col_binned[col_binned == cross.index[i]] = woe
    df_categ = df_categ.drop(c,axis=1)
    df_categ = pd.concat((df_categ, col_binned.rename('WOE_' + c)), axis=1)
df_categ

,WOE_NAME_EDUCATION_TYPE_Higher education,WOE_CODE_GENDER_F,WOE_FLAG_DOCUMENT_3,WOE_FLAG_OWN_CAR_Y,WOE_REGION_RATING_CLIENT_W_CITY,WOE_prev_label_YIELD_mode,WOE_NAME_INCOME_TYPE_Working,WOE_NAME_FAMILY_STATUS_Married,WOE_WALLSMATERIAL_MODE_Panel,WOE_ORGANIZATION_TYPE_Self-employed,WOE_ORGANIZATION_TYPE_Business Entity Type 3,WOE_NAME_CONTRACT_TYPE_Cash loans,WOE_ORGANIZATION_TYPE_Construction,WOE_ORGANIZATION_TYPE_Transport: type 3,WOE_FLAG_DOCUMENT_16,WOE_FLAG_DOCUMENT_18,WOE_FLAG_DOCUMENT_13,WOE_OCCUPATION_TYPE_Security staff,WOE_REG_CITY_NOT_LIVE_CITY,WOE_ORGANIZATION_TYPE_Military,WOE_OCCUPATION_TYPE_Drivers,WOE_OCCUPATION_TYPE_Low-skill Laborers,WOE_OCCUPATION_TYPE_Laborers,WOE_OCCUPATION_TYPE_Sales staff,WOE_ORGANIZATION_TYPE_Industry: type 9,WOE_FLAG_DOCUMENT_14,WOE_WEEKDAY_APPR_PROCESS_START_SATURDAY,WOE_WEEKDAY_APPR_PROCESS_START_MONDAY,WOE_NAME_HOUSING_TYPE_House / apartment,WOE_ORGANIZATION_TYPE_Trade: type 3,WOE_FLAG_DOCUMENT_6,WOE_FLAG_WORK_PHONE,WOE_FLAG_PHONE,WOE_ORGANIZATION_TYPE_Police,WOE_ORGANIZATION_TYPE_Security Ministries
0,-0.111635,0.158120,-0.098265,-0.052305,0.023796,-0.173113,-0.186532,0.069433,-0.063358,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,-0.422258,-0.003466,0.028177,0.008713,0.076457,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,-0.032961,0.051120,-0.05466,-0.00341,-0.002691
1,-0.111635,0.158120,-0.098265,-0.052305,0.023796,0.289740,-0.186532,0.069433,-0.063358,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,0.028177,0.008713,0.076457,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,-0.032961,-0.184029,-0.05466,-0.00341,-0.002691
2,-0.111635,0.158120,0.283560,-0.052305,-0.404551,0.289740,0.238883,0.069433,0.264034,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,0.028177,0.008713,-0.298964,0.026672,-0.002423,-0.002583,-0.007623,0.050117,0.04156,-0.311682,-0.032961,0.051120,-0.05466,-0.00341,-0.002691
3,0.437263,0.158120,-0.098265,-0.052305,-0.404551,0.289740,0.238883,0.069433,-0.063358,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,0.028177,0.008713,0.076457,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,-0.032961,0.051120,-0.05466,-0.00341,-0.002691
4,-0.111635,0.158120,0.283560,-0.052305,-0.404551,-0.034249,0.238883,-0.113213,-0.063358,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,0.028177,0.008713,0.076457,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,0.406590,0.051120,-0.05466,-0.00341,-0.002691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53089,-0.111635,0.158120,-0.098265,0.109776,-0.404551,-0.034249,0.238883,0.069433,-0.063358,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,0.028177,0.008713,0.076457,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,-0.032961,0.051120,-0.05466,-0.00341,-0.002691
53090,-0.111635,-0.258537,-0.098265,0.109776,0.023796,-0.034249,-0.186532,0.069433,-0.063358,0.040891,0.045474,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,0.028177,0.008713,-0.298964,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,-0.032961,-0.184029,-0.05466,-0.00341,-0.002691
53091,-0.111635,-0.258537,-0.098265,-0.052305,0.023796,-0.034249,-0.186532,0.069433,-0.063358,0.040891,-0.146338,-0.034804,0.010787,0.003982,-0.004501,-0.002433,-0.002938,0.009831,0.045215,-0.003466,-0.368362,0.008713,0.076457,0.026672,-0.002423,-0.002583,-0.007623,-0.009725,0.04156,0.003944,-0.032961,-0.184029,0.15183,-0.00341,-0.002691
53092,-0.111635,0.158120,0.283560,0.109776,0.023796,-0.034249,0.238883,0.069433,-0.063358,0.040891,0.045474,-0.034804,0.

In [217]:
# Continuous variables
for c in df_numeric.columns:
    col = df_numeric[c]
    col_binned = pd.cut(col, bins=5, labels = [1,2,3,4,5])
    col_binned = pd.to_numeric(col_binned)
    cross = pd.crosstab(col_binned, y)
    for i in range(len(cross.index)):
        # if some bins have 0 event, use adjust WOE by adding 0.5
        if cross.iloc[i,0] == 0 or cross.iloc[i,1] == 0:
            non_default = (cross.iloc[i,0]+0.5)/cross.iloc[:,0].values.sum()
            default = (cross.iloc[i,1]+0.5)/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        else:
            non_default = cross.iloc[i,0]/cross.iloc[:,0].values.sum()
            default = cross.iloc[i,1]/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        col_binned[col_binned == cross.index[i]] = woe
    df_numeric = df_numeric.drop(c,axis=1)
    df_numeric = pd.concat((df_numeric, col_binned.rename('WOE_' + c)), axis=1)
df_numeric

,WOE_EXT_SOURCE_3,WOE_EXT_SOURCE_2,WOE_EXT_SOURCE_1,WOE_pct_balance_0_1_year,WOE_DAYS_ID_PUBLISH,WOE_DEF_30_CNT_SOCIAL_CIRCLE,WOE_AMT_ANNUITY,WOE_DAYS_LAST_PHONE_CHANGE,WOE_install_default_total,WOE_AMT_REQ_CREDIT_BUREAU_QRT,WOE_prev_GOODS_price_total,WOE_pct_balance_1_2_year,WOE_AMT_GOODS_PRICE,WOE_OWN_CAR_AGE,WOE_DAYS_REGISTRATION,WOE_>30d_default,WOE_CNT_CHILDREN,WOE_avg_DAYS_CREDIT,WOE_FLOORSMAX_AVG
0,-0.057778,0.872285,-0.019414,0.068969,-0.122416,0.0123,-0.035245,0.286152,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,-0.111779,0.000785,0.002894,0.402655,-0.072515
1,-0.057778,-1.038548,-0.019414,0.068969,0.048055,0.0123,-0.035245,-0.165949,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,-0.111779,0.000785,0.002894,0.402655,-0.072515
2,-0.057778,0.290631,1.104975,0.068969,0.048055,0.0123,-0.035245,0.128415,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,-0.111779,0.000785,0.002894,-0.059609,0.327115
3,0.601663,-0.487754,-0.019414,0.068969,0.048055,0.0123,-0.035245,-0.165949,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,-0.111779,0.000785,0.002894,0.402655,-0.072515
4,-0.057778,0.290631,-0.019414,0.068969,0.309422,0.0123,-0.035245,0.128415,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,0.372959,0.000785,0.002894,-0.453591,-0.072515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53089,-1.192869,-0.145973,-0.019414,0.068969,0.048055,0.0123,-0.035245,0.286152,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,-0.111779,0.000785,0.002894,-0.059609,-0.072515
53090,-0.057778,0.290631,-0.019414,-0.688763,0.309422,0.0123,-0.035245,-0.165949,-0.004197,0.000922,0.002224,0.014448,-0.072419,-0.184309,0.090967,0.000785,0.002894,-0.453591,-0.072515
53091,-0.508151,0.290631,-0.908500,-0.688763,0.048055,0.0123,-0.035245,0.286152,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,0.090967,0.000785,0.002894,0.402655,-0.072515
53092,0.601663,0.290631,0.629297,-0.688763,-0.230483,0.0123,-0.035245,0.286152,-0.004197,0.000922,0.002224,0.014448,-0.072419,0.009267,0.090967,0.000785,0.002894,0.402655,-0.072515


WOE transformation completed. Now all features are WOE

In [219]:
# Normalized X and y for logistic regression; train/validation split
X_logit = pd.concat((df_numeric, df_categ), axis=1)
y_logit = y

sc = MinMaxScaler()
X_logit = pd.DataFrame(sc.fit_transform(X_logit), columns=X_logit.columns)
X_train_logit, X_val_logit, y_train_logit, y_val_logit = train_test_split(X_logit, y_logit, test_size=0.3, random_state=42)

Deal with test set

In [221]:
# Select the features form the stepwise selection
test_logit = test[result].copy()

In [222]:
# Do WOE transformation. First split categorical and continous features
index_categ = test_logit.nunique()<7
index_numeric = test_logit.nunique()>=7
df_categ = test_logit[test_logit.columns[index_categ]]
df_numeric = test_logit[test_logit.columns[index_numeric]]

In [223]:
# WOE transformation

# Categorical variables
for c in df_categ.columns:
    col_binned = df_categ[c].copy()
    cross = pd.crosstab(col_binned, y)
    for i in range(len(cross.index)):
        # if some bins have 0 event, use adjust WOE by adding 0.5
        if cross.iloc[i,0] == 0 or cross.iloc[i,1] == 0:
            non_default = (cross.iloc[i,0]+0.5)/cross.iloc[:,0].values.sum()
            default = (cross.iloc[i,1]+0.5)/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        else:
            non_default = cross.iloc[i,0]/cross.iloc[:,0].values.sum()
            default = cross.iloc[i,1]/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        col_binned[col_binned == cross.index[i]] = woe
    df_categ = df_categ.drop(c,axis=1)
    df_categ = pd.concat((df_categ, col_binned.rename('WOE_' + c)), axis=1)
df_categ

,WOE_NAME_EDUCATION_TYPE_Higher education,WOE_CODE_GENDER_F,WOE_FLAG_DOCUMENT_3,WOE_FLAG_OWN_CAR_Y,WOE_REGION_RATING_CLIENT_W_CITY,WOE_prev_label_YIELD_mode,WOE_NAME_INCOME_TYPE_Working,WOE_NAME_FAMILY_STATUS_Married,WOE_WALLSMATERIAL_MODE_Panel,WOE_ORGANIZATION_TYPE_Self-employed,WOE_ORGANIZATION_TYPE_Business Entity Type 3,WOE_NAME_CONTRACT_TYPE_Cash loans,WOE_ORGANIZATION_TYPE_Construction,WOE_ORGANIZATION_TYPE_Transport: type 3,WOE_FLAG_DOCUMENT_16,WOE_FLAG_DOCUMENT_18,WOE_FLAG_DOCUMENT_13,WOE_OCCUPATION_TYPE_Security staff,WOE_REG_CITY_NOT_LIVE_CITY,WOE_ORGANIZATION_TYPE_Military,WOE_OCCUPATION_TYPE_Drivers,WOE_OCCUPATION_TYPE_Low-skill Laborers,WOE_OCCUPATION_TYPE_Laborers,WOE_OCCUPATION_TYPE_Sales staff,WOE_ORGANIZATION_TYPE_Industry: type 9,WOE_FLAG_DOCUMENT_14,WOE_WEEKDAY_APPR_PROCESS_START_SATURDAY,WOE_WEEKDAY_APPR_PROCESS_START_MONDAY,WOE_NAME_HOUSING_TYPE_House / apartment,WOE_ORGANIZATION_TYPE_Trade: type 3,WOE_FLAG_DOCUMENT_6,WOE_FLAG_WORK_PHONE,WOE_FLAG_PHONE,WOE_ORGANIZATION_TYPE_Police,WOE_ORGANIZATION_TYPE_Security Ministries
0,0.004294,-0.000656,-0.003671,0.014601,-0.00194,0.000261,0.025304,0.009492,0.007270,0.002123,-0.043325,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,-0.047887,-0.000234,-0.005001,0.002116,-0.017222,-0.000487,-0.000678
1,0.004294,0.000322,-0.003671,-0.007420,-0.00194,0.000261,0.025304,0.009492,0.007270,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
2,-0.012421,0.000322,-0.003671,-0.007420,-0.00194,0.002295,0.025304,0.009492,0.007270,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
3,0.004294,-0.000656,-0.003671,-0.007420,0.01563,0.000261,-0.024930,0.009492,0.007270,0.002123,-0.043325,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,-0.038008,0.003355,-0.000493,0.0,-0.001421,-0.002072,-0.047887,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
4,0.004294,-0.000656,-0.003671,-0.007420,-0.00194,0.002295,0.025304,0.009492,0.007270,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,0.009945,0.005604,-0.000234,-0.005001,-0.008217,0.006154,-0.000487,-0.000678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0.004294,0.000322,-0.003671,-0.007420,-0.00194,0.000261,0.025304,0.009492,0.007270,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
48740,-0.012421,0.000322,-0.003671,-0.007420,0.01563,-0.002501,-0.024930,0.009492,0.007270,0.002123,0.012412,-0.00016,-3.752361e-05,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,-0.038008,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
48741,-0.012421,-0.000656,-0.003671,-0.007420,-0.00194,0.000261,-0.024930,-0.018598,0.007270,-0.015350,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,0.019423,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
48742,0.004294,0.000322,-0.003671,-0.007420,-0.00194,0.002295,-0.024930,-0.018598,0.007270,-0.015350,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217

In [224]:
# Continuous variables
for c in df_numeric.columns:
    col = df_numeric[c]
    col_binned = pd.cut(col, bins=5, labels = [1,2,3,4,5])
    col_binned = pd.to_numeric(col_binned)
    cross = pd.crosstab(col_binned, y)
    for i in range(len(cross.index)):
        # if some bins have 0 event, use adjust WOE by adding 0.5
        if cross.iloc[i,0] == 0 or cross.iloc[i,1] == 0:
            non_default = (cross.iloc[i,0]+0.5)/cross.iloc[:,0].values.sum()
            default = (cross.iloc[i,1]+0.5)/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        else:
            non_default = cross.iloc[i,0]/cross.iloc[:,0].values.sum()
            default = cross.iloc[i,1]/cross.iloc[:,1].values.sum()
            woe = np.log(non_default/default)
        col_binned[col_binned == cross.index[i]] = woe
    df_numeric = df_numeric.drop(c,axis=1)
    df_numeric = pd.concat((df_numeric, col_binned.rename('WOE_' + c)), axis=1)
df_numeric

,WOE_EXT_SOURCE_3,WOE_EXT_SOURCE_2,WOE_EXT_SOURCE_1,WOE_pct_balance_0_1_year,WOE_DAYS_ID_PUBLISH,WOE_DEF_30_CNT_SOCIAL_CIRCLE,WOE_AMT_ANNUITY,WOE_DAYS_LAST_PHONE_CHANGE,WOE_install_default_total,WOE_AMT_REQ_CREDIT_BUREAU_QRT,WOE_prev_GOODS_price_total,WOE_pct_balance_1_2_year,WOE_AMT_GOODS_PRICE,WOE_OWN_CAR_AGE,WOE_DAYS_REGISTRATION,WOE_>30d_default,WOE_CNT_CHILDREN,WOE_avg_DAYS_CREDIT,WOE_FLOORSMAX_AVG
0,0.013396,-0.006522,0.013006,0.001289,0.010383,-0.000033,0.000253,0.001964,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.004270,-0.00043,-0.000105,0.004458,0.007190
1,0.013396,-0.006522,0.009975,-0.012961,0.028604,-0.000033,0.000253,0.007968,0.000588,-0.006132,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.004458,0.007190
2,-0.008848,-0.022879,0.009975,0.001289,-0.025292,-0.000033,0.000253,-0.014157,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.031968,0.007190
3,0.012316,0.049536,0.013006,0.001289,0.028604,-0.000033,0.000253,0.001964,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.003462,0.007190
4,0.013396,-0.011213,-0.040966,0.060138,0.010383,-0.000033,0.000253,0.001964,0.000588,-0.006132,0.00126,-0.000435,-0.001096,-0.005316,-0.004270,-0.00043,-0.000105,0.004458,0.007190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,-0.008848,-0.011213,0.013006,0.001289,0.010383,-0.000033,0.000253,0.007968,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,0.009491,-0.00043,-0.000105,0.004458,0.007190
48740,0.013396,0.049536,-0.040966,0.001289,-0.025292,-0.000033,-0.001379,-0.014157,0.000588,0.002565,0.00126,-0.000435,0.001290,-0.005316,-0.004270,-0.00043,-0.000105,0.004458,0.007190
48741,-0.012551,0.049536,0.013006,0.001289,0.028604,-0.000033,0.000253,0.007968,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.004458,0.007190
48742,-0.012551,-0.022879,-0.026387,0.001289,-0.036935,-0.000033,0.000253,0.001964,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.004458,0.007190


In [226]:
# Normalize the test set for logistic regression
test_logit = pd.concat((df_numeric, df_categ), axis=1)

sc = MinMaxScaler()
test_logit = pd.DataFrame(sc.fit_transform(test_logit), columns=test_logit.columns)
test_logit.head()

,WOE_EXT_SOURCE_3,WOE_EXT_SOURCE_2,WOE_EXT_SOURCE_1,WOE_pct_balance_0_1_year,WOE_DAYS_ID_PUBLISH,WOE_DEF_30_CNT_SOCIAL_CIRCLE,WOE_AMT_ANNUITY,WOE_DAYS_LAST_PHONE_CHANGE,WOE_install_default_total,WOE_AMT_REQ_CREDIT_BUREAU_QRT,WOE_prev_GOODS_price_total,WOE_pct_balance_1_2_year,WOE_AMT_GOODS_PRICE,WOE_OWN_CAR_AGE,WOE_DAYS_REGISTRATION,WOE_>30d_default,WOE_CNT_CHILDREN,WOE_avg_DAYS_CREDIT,WOE_FLOORSMAX_AVG,WOE_NAME_EDUCATION_TYPE_Higher education,WOE_CODE_GENDER_F,WOE_FLAG_DOCUMENT_3,WOE_FLAG_OWN_CAR_Y,WOE_REGION_RATING_CLIENT_W_CITY,WOE_prev_label_YIELD_mode,WOE_NAME_INCOME_TYPE_Working,WOE_NAME_FAMILY_STATUS_Married,WOE_WALLSMATERIAL_MODE_Panel,WOE_ORGANIZATION_TYPE_Self-employed,WOE_ORGANIZATION_TYPE_Business Entity Type 3,WOE_NAME_CONTRACT_TYPE_Cash loans,WOE_ORGANIZATION_TYPE_Construction,WOE_ORGANIZATION_TYPE_Transport: type 3,WOE_FLAG_DOCUMENT_16,WOE_FLAG_DOCUMENT_18,WOE_FLAG_DOCUMENT_13,WOE_OCCUPATION_TYPE_Security staff,WOE_REG_CITY_NOT_LIVE_CITY,WOE_ORGANIZATION_TYPE_Military,WOE_OCCUPATION_TYPE_Drivers,WOE_OCCUPATION_TYPE_Low-skill Laborers,WOE_OCCUPATION_TYPE_Laborers,WOE_OCCUPATION_TYPE_Sales staff,WOE_ORGANIZATION_TYPE_Industry: type 9,WOE_FLAG_DOCUMENT_14,WOE_WEEKDAY_APPR_PROCESS_START_SATURDAY,WOE_WEEKDAY_APPR_PROCESS_START_MONDAY,WOE_NAME_HOUSING_TYPE_House / apartment,WOE_ORGANIZATION_TYPE_Trade: type 3,WOE_FLAG_DOCUMENT_6,WOE_FLAG_WORK_PHONE,WOE_FLAG_PHONE,WOE_ORGANIZATION_TYPE_Police,WOE_ORGANIZATION_TYPE_Security Ministries
0,0.013396,-0.006522,0.013006,0.001289,0.010383,-0.000033,0.000253,0.001964,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.004270,-0.00043,-0.000105,0.004458,0.00719,0.004294,-0.000656,-0.003671,0.014601,-0.00194,0.000261,0.025304,0.009492,0.00727,0.002123,-0.043325,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,-0.047887,-0.000234,-0.005001,0.002116,-0.017222,-0.000487,-0.000678
1,0.013396,-0.006522,0.009975,-0.012961,0.028604,-0.000033,0.000253,0.007968,0.000588,-0.006132,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.004458,0.00719,0.004294,0.000322,-0.003671,-0.007420,-0.00194,0.000261,0.025304,0.009492,0.00727,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
2,-0.008848,-0.022879,0.009975,0.001289,-0.025292,-0.000033,0.000253,-0.014157,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.031968,0.00719,-0.012421,0.000322,-0.003671,-0.007420,-0.00194,0.002295,0.025304,0.009492,0.00727,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,-0.002072,0.005604,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
3,0.012316,0.049536,0.013006,0.001289,0.028604,-0.000033,0.000253,0.001964,0.000588,0.002565,0.00126,-0.000435,-0.001096,-0.005316,-0.002167,-0.00043,-0.000105,0.003462,0.00719,0.004294,-0.000656,-0.003671,-0.007420,0.01563,0.000261,-0.024930,0.009492,0.00727,0.002123,-0.043325,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,-0.038008,0.003355,-0.000493,0.0,-0.001421,-0.002072,-0.047887,-0.000234,-0.005001,0.002116,0.006154,-0.000487,-0.000678
4,0.013396,-0.011213,-0.040966,0.060138,0.010383,-0.000033,0.000253,0.001964,0.000588,-0.006132,0.00126,-0.000435,-0.001096,-0.005316,-0.004270,-0.00043,-0.000105,0.004458,0.00719,0.004294,-0.000656,-0.003671,-0.007420,-0.00194,0.002295,0.025304,0.009492,0.00727,0.002123,0.012412,-0.00016,8.172534e-07,-0.000349,0.0,0.000786,0.0,-0.000933,0.002271,0.000717,-0.001171,-0.000412,0.008217,0.003355,-0.000493,0.0,-0.001421,0.009945,0.005604,-0.000234,-0.005001,-0.008217,0.006154,-0.000487,-0.000678


# Model Fitting and Selection
- We are gonna fit the base model, XGBoost model and logistic regression model
- hyperparameter tuning, k-folds cross-validation for model evaluation
- Chooose the best model to predict for the test set